## Concept Drift in Regression Data

In [1]:
# import the functions needed for validate and comparsion

import numpy as np
import cupy as cp
import pandas as pd
from timeit import default_timer as timer
import os

from sklearn.model_selection import train_test_split
#mean_squared_error: mse squared=true, rmse squared=false
from sklearn.metrics import mean_squared_error
from skmultiflow.data import ConceptDriftStream, FileStream
from sklearn.linear_model import SGDRegressor
from skmultiflow.data.regression_generator import RegressionGenerator
from sklearn.preprocessing import MinMaxScaler


# using plotly for plots
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
from fires import FIRES

In [3]:
# stability measurment as proposed in "Measurment the Stability of Feature Selection"

# TODO: check for case where nothing changes


def pearson_stability_ij(arr1,arr2):
    d = len(arr1)
    k_i = np.sum(arr1)
    k_j = np.sum(arr2)

    # catch edge cases as proposed in the paper under 4.1
    if (k_i == 0 or k_i == d) and k_i != k_j :
        return 0
    elif (k_j == 0 or k_j == d) and k_i != k_j :
        return 0
    elif (k_i == 0 or k_i == d) and k_i == k_j :
        return 1
    x_hat_i = k_i / d
    x_hat_j = k_j / d
    arr1 = arr1 - x_hat_i
    arr2 = arr2 - x_hat_j
    dividend = 1/d * np.sum(arr1*arr2)
    divisor = np.sqrt(1/d*np.sum(arr1**2))*np.sqrt(1/d*np.sum(arr2**2))
    return dividend/divisor

def stability_factor(selected_ftrs):
   M = len(selected_ftrs)
   sum_stabilities = 0
   for i in range(M):
       for j in range(i+1, M):
           sum_stabilities += pearson_stability_ij(selected_ftrs[i], selected_ftrs[j])
   return 1/(M*(M-1))*sum_stabilities * 2   

In [4]:
stream = ConceptDriftStream(stream=RegressionGenerator(random_state=42, n_features=100, n_informative=25), drift_stream=RegressionGenerator(random_state=52, n_features=100, n_informative=25), position=10100, width=10)

dataset_name = "Concept_drift_10"
n_selected_ftr = 25
n_window = 100
batch_size = 10

In [16]:

stream = ConceptDriftStream(stream=RegressionGenerator(random_state=42, n_features=100, n_informative=25), drift_stream=RegressionGenerator(random_state=52, n_features=100, n_informative=25), position=10100, width=1000)

dataset_name = "Concept_drift_1000"
n_selected_ftr = 25
n_window = 100
batch_size = 10

In [17]:
# prepare folder for plots
folder = "plots/regression/{}".format(dataset_name)
if not os.path.exists(folder):
    os.makedirs(folder)

export_type = "pdf" # "png", "jpeg", "webp", "pdf", "svg"

In [18]:
# use SGDRegressor as predictor
stream.restart()
predictor = SGDRegressor()
scaler = MinMaxScaler()
scaler_y = MinMaxScaler()
X, y = stream.next_sample(batch_size=100)
scaler.partial_fit(X)
scaler_y.partial_fit(y.reshape(-1,1))

X = scaler.transform(X)
y = scaler_y.transform(y.reshape(-1,1))
predictor.partial_fit(X,y)

SGDRegressor()

In [19]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=None,
                    mu_init=0,
                    sigma_init=1,
                    model='regression')  

In [20]:

fires_rmse = []


fires_selected_ftrs = []
fires_stability = []

start_time_all = timer()
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=10)
    scaler.partial_fit(x)
    scaler_y.partial_fit(y.reshape(-1,1))
    x = scaler.transform(x)
    y = scaler_y.transform(y.reshape(-1,1))
    # Select features
    ftr_weights = fires_model.weigh_features(x,y)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fires_selected_ftrs.append(ftr_array)

    if len(fires_selected_ftrs) >= 10:
        stability = stability_factor(fires_selected_ftrs[-10:])
        fires_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    fires_rmse.append(mean_squared_error(y, y_pred, squared=False))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
fires_rmse = pd.Series(fires_rmse).rolling(window=n_window).mean().iloc[n_window-1:].values
end_time_all = timer()
fires_run_time = timer() - start_time_all
print("The whole fires run took {}".format(fires_run_time))

The whole fires run took 24.179165686000488


In [21]:
ftr_weights

array([0.28177261, 0.96948851, 0.75020323, 0.65041491, 0.91796415,
       0.37109114, 0.5859747 , 0.38308674, 0.53968765, 0.98618914,
       0.41466499, 1.        , 0.13724451, 0.46413773, 0.62450856,
       0.44808479, 0.31407879, 0.67502176, 0.6483979 , 0.62676318,
       0.51999616, 0.47457979, 0.54096894, 0.54852285, 0.79483174,
       0.10956979, 0.56836345, 0.32595203, 0.06573833, 0.76942251,
       0.55745764, 0.70710102, 0.5463732 , 0.76071624, 0.49491914,
       0.48363249, 0.37232481, 0.79378546, 0.42412727, 0.66604993,
       0.20951138, 0.44535885, 0.51858653, 0.01804131, 0.58343431,
       0.45312293, 0.23324394, 0.6438096 , 0.37735994, 0.24930673,
       0.52112247, 0.26951907, 0.53382172, 0.        , 0.44221606,
       0.77381893, 0.5947587 , 0.15758443, 0.60156397, 0.73986285,
       0.18210503, 0.68034603, 0.82631871, 0.69386959, 0.74272348,
       0.48183718, 0.53000329, 0.75807349, 0.60201524, 0.66332068,
       0.66180211, 0.4562128 , 0.30426858, 0.99332728, 0.14441

In [22]:
# use SGDRegressor as predictor
stream.restart()
predictor = SGDRegressor()
scaler = MinMaxScaler()
scaler_y = MinMaxScaler()
X, y = stream.next_sample(batch_size=100)
scaler.partial_fit(X)
scaler_y.partial_fit(y.reshape(-1,1))

X = scaler.transform(X)
y = scaler_y.transform(y.reshape(-1,1))
predictor.partial_fit(X,y)

SGDRegressor()

In [23]:
sgdr_model = SGDRegressor(penalty="l1") #penalty could be elasticnet as well
#n_selectey_ftrs?

In [24]:
sgdr_rmse = []

sgdr_selected_ftrs = []
sgdr_stability = []

start_time_all = timer()
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=10)
    scaler.partial_fit(x)
    scaler_y.partial_fit(y.reshape(-1,1))
    x = scaler.transform(x)
    y = scaler_y.transform(y.reshape(-1,1))
    # Select features
    sgdr_model.partial_fit(x,y)
    ftr_weights = sgdr_model.coef_
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    sgdr_selected_ftrs.append(ftr_array)

    if len(sgdr_selected_ftrs) >= 10:
        stability = stability_factor(sgdr_selected_ftrs[-10:])
        sgdr_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    sgdr_rmse.append(mean_squared_error(y, y_pred, squared=False))
    
    


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
sgdr_rmse = pd.Series(sgdr_rmse).rolling(window=n_window).mean().iloc[n_window-1:].values
end_time_all = timer()
sgdr_run_time = timer() - start_time_all
print("The whole sgdr run took {}".format(sgdr_run_time))


The whole sgdr run took 11.771736005999628


In [25]:
# stability
title = "Stability on dataset {}".format(dataset_name)
col_names = ["FIRES", "SGDR"]
d = {"FIRES":fires_stability, "SGDR":sgdr_stability}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y = col_names, title=title, labels={"index":"batches", "value":"stability"}, color_discrete_map={'FIRES': 'red', "SGDR": "blue"})
stability_trace = fig["data"]
fig.show()

In [26]:
title = "RMSE on dataset {}".format(dataset_name)
col_names = ["FIRES", "SGDR"]
d = {"FIRES":fires_rmse, "SGDR":sgdr_rmse}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y = col_names, title=title, labels={"index":"batches", "value":"rmse"}, color_discrete_map={'FIRES': 'red', "SGDR": "blue"})
rmse_trace = fig["data"]
fig.show()

In [27]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1) # subplot_titles=("Stability","Accuracy", "F1-Score"),
for i in range(len(stability_trace)):
    stability_trace[i]["showlegend"] = False
    fig.add_trace(stability_trace[i], row=1, col=1)
for i in range(len(rmse_trace)):
    fig.add_trace(rmse_trace[i], row=2, col=1)
fig.update_xaxes(title_text="batches", row=2, col=1)
fig.update_yaxes(title_text="Stability", row=1, col=1)
fig.update_yaxes(title_text="RMSE", row=2, col=1)

fig.write_image("{}/{}.{}".format(folder, dataset_name, export_type))
fig.show()